# Tutorial 2: Construct MDFPs from MD Simulations

## Outline 

0. Useful Information before starting <br>


1. Construct MDFPs <br>
    1.1 Construct the MDFP for a single compound <br>
    1.2 Construct the MDFPs from all the trajectoriy files in a given folder <br>
    1.3 Construct MDFPs from python shell <br>
    
    
2. Customize MDFPs <br>
    2.1 Customize energy terms <br>
    2.2 Add different features <br>


## 0. Useful Information before starting 

#### I)   INPUT FILES:
MD trajectory (.xtc, .h5, .trr), coordinates file (.gro or .pdb), and file containing energy terms extracted from the trajectories (.xvg or .csv).

To compute 2D counts and proeperties, also SDF files or SMILES of the compounds are required. 

#### II) REQUIRED SOFTWARES AND PYTHON LIBRARIES

##### Softwares:
python3 <br>

##### Python Liraries:
RDKit <br>
subprocess<br>
argparse <br>
pandas <br>
numpy <br>
glob <br>
mdtraj <br>
pymol <br>
os <br>
scipy <br>
(parmed)


#### II)   CHECK SCRIPTS USAGE:

        python3 script.py -h



## 1. Construct MDFPs

## 1.1 Construct the MDFP for a single compound 

To compute the MDFP for a single compound, use the script compose_mdfp.py

To print out the help message:

This returns:

    usage: compose_mdfp.py [-h] [-traj_file traj.xtc] [-coord_file coord.gro]
                           [-energy_file energy.xvg] [-sdf_file compound.sdf]
                           [-ismi SMILES] [-cmpd_name compound_ID]
                           [-output_filename output]
                           [-output_folder OUTPUT_FOLDER]

    DESCRIPTION: 
    ---------------- 
    Compose MDFP from a MD trajectory. 
    It returns a dataframe (pkl file) containing the MDFP terms and 2D-properties of 
    the analyzed compound. 
    To compute MD terms, the arguments needed are: -xtc_file, -gro_file, -top_file, 
    and -energy_file. 
    To compute 2D count and properties, specify -sdf_file or -ismi.
 
     ARGUMENTS 
     ---------------- 

    optional arguments:
      -h, --help            show this help message and exit
      -traj_file traj.xtc   Trajectory file (.xtc, .trr, .h5)
      -coord_file coord.gro 
                            Coordinates file (.gro, .pdb)
      -energy_file energy.xvg
                            File containing the energy terms extracted from the 
                            simulations (.xvg)
      -sdf_file compound.sdf
                            SDF file of the compound. Used to compute topological counts 
                            and properties. Alternative to -ismi
      -ismi SMILES          SMILES of the compound. Used to compute topological counts and 
                            properties. Alternative to -sdf_file.
      -cmpd_name compound_ID
                            Name of the compound being analyzed. Returned in the output 
                            dataframe if specified
      -output_filename output
                            If specified, the output file name is "MDFP_output.pkl". 
                            Otherwise the output file is "MDFP.pkl" or "MDFP_cmpd_name.pkl" 
                            if -cmpd_name is specified)
      -output_folder OUTPUT_FOLDER
                            Folder in which the output file is written (Default = ".")


The traj_file (.xtc) and energy_file (.xvg) are generated by running the MD simulations. Therefore, by running the script run_md_water_4tutorial1.sh of Tutorial1. 
The energy_file.xvg can be customized according to the MD simulations or according to the energy terms of interest. An example is provided below.

The coord_file (.gro or .pdb) is a necessary input files to start MD simulations.  

To compute topological counts and properties, either the SDF file or the SMILES of the compounds must be given as input. The SDF files are generated in step 1 of Tutorial1 and stored in the folder sdf_files.

To compute the MDFP for CHEMBL1077779, run the following command. The output MDFP is stored in a dataframe (MDFP_CHEMBL1077779.pkl).

## 1.2 Construct the MDFPs from all the trajectoriy files in a given folder

To extract the MDFP from multiple trajectory files, all contained in the same folder, you can use the script compose_mdfp_from_folder.py.

Print help message:

This returns:

    usage: compose_mdfp_from_folder.py [-h] -traj_folder path [-sdf_folder path]
                                       [-output_filename output]
                                       [-output_folder path] [-save_steps integer]

    DESCRIPTION: 
    ---------------- 
     Compose MDFP from all the MD trajectories (.xtc) in the folder provided as input (-traj_folder). 
     It returns a dataframe (pkl file) containing the MDFP terms and 2D-properties for all the analyzed compounds. 
     The name of the compound is guessed from the trajectory filename.  
     Other required files for running this script (.gro and .avg) are searched in the folder provided as input (-traj_folder) based on the guessed compound name. 
     To compute 2D count and properties, specify -sdf_folder. 
 
    ARGUMENTS 
    ---------------- 

    optional arguments:
      -h, --help            show this help message and exit
      -traj_folder path     folder containing MD files
      -sdf_folder path      folder containing SDF files
      -output_filename output
                            If specified, the output file is "MDFP_output.pkl". Otherwise 
                            the output file is "MDFP.pkl". If MDFP.pkl already exists in
                            the output folder, it will be overwritten
      -output_folder path   folder in which the output files are written
      -save_steps integer   Saving frequency. Save the MDFPs every 10 compounds being 
                            analyzed

## 1.3 Construct MDFPs from python shell 

Import required libraries

In [1]:
import sys
sys.path.append('ex2_compose_mdfp')
from ComposerGmx import *

Read required input files: trajectory file, coordinates file, file containing energy terms, and SDF file

In [2]:
traj_file = 'ex2_compose_mdfp/data/center_nvt3_CHEMBL1077779_pH7_netcharge.xtc'
energy_file = 'ex2_compose_mdfp/data/nvt3_rf_CHEMBL1077779_pH7_netcharge.xvg'
coord_file = 'ex2_compose_mdfp/data/CHEMBL1077779_pH7_netcharge.gro'
sdf_file = 'ex2_compose_mdfp/sdf_files/CHEMBL1077779_pH7.sdf'

Initialize output dictionary in which to store MDFP terms

In [3]:
Fingerprint = [] # initialize list of dictionaries. one for each compound
Dict_Fingerprint = {} # initialize dictionary

Add name of the compound to the dictionary

In [4]:
Dict_Fingerprint.update({'cmpd_name': 'CHEMBL1077779'})

Calculate 2D counts and properties:

In [5]:
# Read in molecule
mol = Chem.MolFromMolFile(sdf_file)
smiles = Chem.MolToSmiles(mol)
Dict_Fingerprint.update({"smiles": smiles})
# Compute 2D counts and properties and add them to the dictionary
Dict_Fingerprint.update(ComposerGMX.Properties2D_From_Mol(mol))
# 2D counts and properties can also be computed from SMILES using the functions ComposerGMX.Properties2D_From_SMILES(SMILES)
Dict_Fingerprint

{'cmpd_name': 'CHEMBL1077779',
 'smiles': 'CCCCC1N(CC2CCCCC2)C(=O)OC12CC[NH+](C1(C)CCN(C(=O)c3c(C)cccc3C)CC1)CC2',
 'MW': 538.40031894409,
 'HA_count': 39,
 'RB_count': 8,
 'N_count': 3,
 'O_count': 3,
 'F_count': 0,
 'P_count': 0,
 'S_count': 0,
 'Cl_count': 0,
 'Br_count': 0,
 'I_count': 0,
 'HBD_count': 1,
 'HBA_count': 5,
 '2d_shape': 6.4040808072224715,
 '2d_psa': 0.5429,
 'is_zwit': 0}

Compute MDFP terms:

To check the documentation of each function type:

In [6]:
help(ComposerGMX.Properties2D_From_Mol)

Help on method Properties2D_From_Mol in module ComposerGmx:

Properties2D_From_Mol(molecule, cmpd_name=None, includeSandP=None, dist_cutoff=None) method of builtins.type instance
    It returns a dictionary of 2D-counts and properties for the input RDKit molecule. 
    These are: number of heavy atoms ("HA_count"), number of rotatable bonds ("RB_count"), number of N, O, F, P, S, Cl, Br, and I atoms ("ATOM_count" where ATOM is the element), molecular weight ("MW"), number of hydrogen bond donor ("HBD_count") and acceptor atoms ("HBA_count"), binary zwitterionic flag ("is_zwit"), 2D-shape parameter ("2d_shape"), and the topological polar surface area ("2d_psa"). 
    The 2D-shape parameter descibes the elongation of the molecule (see the function calc_shape_from_Mol for details).
    
    Parameters:
    ----------
    Molecule: RDKit molecule
        Molecule read in using one of the RDKit functions. See RDKit documentation.
    includeSandP: bool, optional
        Set to False to exclu

In [7]:
# Compute energy terms from the energy_file and add them to the dictionary
Dict_Fingerprint.update(ComposerGMX.EnergyTermsGmx(energy_file))

# The following steps are required to read in the trajectory and compute sasa and rgyr
# convert gro to pdb
ComposerGMX.gro2pdb(coord_file)
# the output pdb file will have the same name of the gro file: CHEMBL1077779_pH7_netcharge.pdb
pdb_file = os.path.splitext(coord_file)[0] + '.pdb'
# load pdb and identify indices of solute atoms
pdb = md.load(pdb_file)
topology = pdb.topology
solute_atoms = ComposerGMX.solute_solvent_split(topology)[0]
# read in the trajectory but only for the solute. Solvent not loaded
solute_traj = md.load(traj_file, top=pdb_file, atom_indices = solute_atoms)
# Compute SASA and Rgyr and add the terms to the output dictionary
Dict_Fingerprint.update(ComposerGMX.extract_rgyr(solute_traj))
Dict_Fingerprint.update(ComposerGMX.extract_sasa(solute_traj))

# Compute the 3D-PSA. The extract_psa3d function uses pymol to compute the 3D-PSA.
Dict_Fingerprint.update(ComposerGMX.extract_psa3d(traj_file, coord_file, include_SandP = True))

 PyMOL not running, entering library mode (experimental)


In [8]:
Dict_Fingerprint

{'cmpd_name': 'CHEMBL1077779',
 'smiles': 'CCCCC1N(CC2CCCCC2)C(=O)OC12CC[NH+](C1(C)CCN(C(=O)c3c(C)cccc3C)CC1)CC2',
 'MW': 538.40031894409,
 'HA_count': 39,
 'RB_count': 8,
 'N_count': 3,
 'O_count': 3,
 'F_count': 0,
 'P_count': 0,
 'S_count': 0,
 'Cl_count': 0,
 'Br_count': 0,
 'I_count': 0,
 'HBD_count': 1,
 'HBA_count': 5,
 '2d_shape': 6.4040808072224715,
 '2d_psa': 0.5429,
 'is_zwit': 0,
 'intra_crf_av_wat': -362.0758371761648,
 'intra_crf_std_wat': 4.52381883701882,
 'intra_crf_med_wat': -362.216736,
 'intra_lj_av_wat': 67.78596650349931,
 'intra_lj_std_wat': 10.158554294182972,
 'intra_lj_med_wat': 67.22666999999998,
 'total_crf_av_wat': -655.1365829774045,
 'total_crf_std_wat': 28.993538457079023,
 'total_crf_med_wat': -655.15396,
 'total_lj_av_wat': -122.93220517016599,
 'total_lj_std_wat': 17.238890131060966,
 'total_lj_med_wat': -123.475686,
 'intra_ene_av_wat': -294.2898706726655,
 'intra_ene_std_wat': 10.655169132970629,
 'intra_ene_med_wat': -294.86599299999995,
 'total_en

Convert dictionary to dataframe:

In [9]:
Fingerprint.append(Dict_Fingerprint) # append one dictionary for every compound analyzed.
pd.DataFrame(Fingerprint)

,cmpd_name,smiles,MW,HA_count,RB_count,N_count,O_count,F_count,P_count,S_count,...,total_ene_med_wat,wat_rgyr_av,wat_rgyr_std,wat_rgyr_med,wat_sasa_av,wat_sasa_std,wat_sasa_med,3d_psa_av,3d_psa_sd,3d_psa_med
0,CHEMBL1077779,CCCCC1N(CC2CCCCC2)C(=O)OC12CC[NH+](C1(C)CCN(C(...,538.400319,39,8,3,3,0,0,0,...,-778.144234,0.572236,0.014888,0.572389,8.788033,0.100725,8.794202,0.577738,0.011521,0.577808


## 2. Customize MDFPs

## 2.1 Customize energy terms 

The energy terms are extracted from the MD trajectories using the GROMACS function gmx energy. See also GROMACS documentation (http://manual.gromacs.org/documentation/current/onlinehelp/gmx-energy.html).

You can compute different energy terms by running the command below. The specified terms are computed for each frame and stored in the specified .xvg file.

Once you have the xvg file, you need to write your own classmethod in the ComposerGMX class (in the file classes_composer.py) or, alternatively, you can write your own function in one of the compose_mdfp python scripts. As an example, look at the EnergyTermsGmx classmethod in classes_composer.py.  

The first line of your function/classmethod should call the read_xvg function:

In [10]:
# import libraries 
import sys
sys.path.append('ex2_compose_mdfp')
from classes_composer import *

energy_df = ComposerGMX.read_xvg("ex2_compose_mdfp/data/nvt3_rf_CHEMBL1077779_pH7_netcharge.xvg")
energy_df.head()
    

,time,Coul-SR:Water_and_ions-LIG,LJ-SR:Water_and_ions-LIG,Coul-14:Water_and_ions-LIG,LJ-14:Water_and_ions-LIG,Coul-SR:LIG-LIG,LJ-SR:LIG-LIG,Coul-14:LIG-LIG,LJ-14:LIG-LIG
0,0.0,-248.006775,-193.824005,0.0,0.0,293.304169,-37.128689,-658.616028,96.577568
1,1.0,-326.848846,-188.927216,0.0,0.0,288.573090,-23.830988,-653.050781,110.039108
2,2.0,-243.550308,-189.362839,0.0,0.0,285.984253,-26.193317,-653.422607,101.753799
3,3.0,-288.854401,-201.806198,0.0,0.0,287.162292,-39.007591,-653.411926,103.558929
4,4.0,-305.545929,-177.565567,0.0,0.0,282.829010,-40.269783,-655.066284,105.343872


This function reads the xvg file into a dataframe. The column names will contain the names of the energy terms as defined by GROMACS. 

Alternative, if you already have the energy terms in a dataframe, you might want to use functions of pandas to read the data in. For example, for a csv file use pd.read_csv(energy_file). 

If you want to add an energy term which is the sum of all the LJ contributions, do:

In [11]:
#look for all the columns having the 'LJ' word in the column name 
LJ_column_indeces = [i for i, s in enumerate(energy_df) if 'LJ' in s]
print("The column containing LJ energy contributions are: {}".format(LJ_column_indeces))

# Convert the dataframe into a numpy array
energy_array = np.array(energy_df)

# sum up all the columns containing LJ energy contributions
lj = np.sum(energy_array[:,LJ_column_indeces], axis=1)

# calculate mean, standard deviation and median
lj_mean, lj_std, lj_med = ComposerGMX.get_stats(lj)
print("Mean, standard deviation and median of the LJ energy term are:{},{},and {} kJ/mol".format(lj_mean, lj_std, lj_med))

# add newly computed terms to a dictionary
dict_ene_new = {'lj_mean': lj_mean, 'lj_std':lj_std, 'lj_med':lj_med}

# all terms composing the MDFP are included in the dictionary Dict_Fingerprint. Update the dictionary:
Dict_Fingerprint.update(dict_ene_new)

The column containing LJ energy contributions are: [2, 4, 6, 8]
Mean, standard deviation and median of the LJ energy term are:-122.93220517016599,17.238890131060963,and -123.47568600000001 kJ/mol


Define a function that executes all those steps:

In [12]:
def Customized_EnergyTermsGmx(energy_file_xvg):
    # read energy file. The ComposerGMX.read_xvg function reads the xvg file into a dataframe. 
    # The column names will contain the names of the energy terms as defined by GROMACS.
    energy_df = ComposerGMX.read_xvg("ex2_compose_mdfp/data/nvt3_rf_CHEMBL1077779_pH7_netcharge.xvg")
    energy_df.head()
    # Convert the dataframe into a numpy array
    energy_array = np.array(energy_df)
    # write extracted terms into a dictionary
    LJ_column_indeces = [i for i, s in enumerate(energy_df) if 'LJ' in s]
    energy_array = np.array(energy_df)
    # Sum the energy terms
    lj = np.sum(energy_array[:,LJ_column_indeces], axis=1)
    # calculate mean, standard deviation and median
    lj_mean, lj_std, lj_med = ComposerGMX.get_stats(lj)
    # add newly computed terms to a dictionary
    dict_ene_new = {'lj_mean': lj_mean, 'lj_std':lj_std, 'lj_med':lj_med}
    return(dict_ene_new)

Execute the fuction and update the Dict_Fingerprint dictionary:

In [13]:
energy_file = "ex2_compose_mdfp/data/nvt3_rf_CHEMBL1077779_pH7_netcharge.xvg"
Dict_Fingerprint.update(Customized_EnergyTermsGmx(energy_file))
Dict_Fingerprint

{'cmpd_name': 'CHEMBL1077779',
 'smiles': 'CCCCC1N(CC2CCCCC2)C(=O)OC12CC[NH+](C1(C)CCN(C(=O)c3c(C)cccc3C)CC1)CC2',
 'MW': 538.40031894409,
 'HA_count': 39,
 'RB_count': 8,
 'N_count': 3,
 'O_count': 3,
 'F_count': 0,
 'P_count': 0,
 'S_count': 0,
 'Cl_count': 0,
 'Br_count': 0,
 'I_count': 0,
 'HBD_count': 1,
 'HBA_count': 5,
 '2d_shape': 6.4040808072224715,
 '2d_psa': 0.5429,
 'is_zwit': 0,
 'intra_crf_av_wat': -362.0758371761648,
 'intra_crf_std_wat': 4.52381883701882,
 'intra_crf_med_wat': -362.216736,
 'intra_lj_av_wat': 67.78596650349931,
 'intra_lj_std_wat': 10.158554294182972,
 'intra_lj_med_wat': 67.22666999999998,
 'total_crf_av_wat': -655.1365829774045,
 'total_crf_std_wat': 28.993538457079023,
 'total_crf_med_wat': -655.15396,
 'total_lj_av_wat': -122.93220517016599,
 'total_lj_std_wat': 17.238890131060966,
 'total_lj_med_wat': -123.475686,
 'intra_ene_av_wat': -294.2898706726655,
 'intra_ene_std_wat': 10.655169132970629,
 'intra_ene_med_wat': -294.86599299999995,
 'total_en

## 2.2 Add different features 

It is very simple to calculate properties different from those compared computed by default. Only 3 steps are required:

    1. Compute the property of interest for every trajectory frame. There are different python packages containing MD analysis tools, such as MDTraj, pytraj, MDanalysis,...
    2. Use the function ComposerGMX.get_stat() to obtain mean, standard deviation and median. 
    3. Store the terms of interest in the output dictionary. 

Let's assume you want to add to the MDFP the mean, standard deviation, and median of the dipole moment and of the dipole moment x-, y-, z- components. 

To do this, the partial charges of the solute atoms are needed. These are contained in the topology file (.top for GROMACS). The topology file can be read using the parmed library.


In [14]:
import parmed as pmd

# read topology file
top_file = "ex2_compose_mdfp/data/CHEMBL1077779_pH7_netcharge.top"
prm_obj = pmd.gromacs.GromacsTopologyFile(top_file)

# extract partial charges of the solute atoms
solute_charges = [i.charge for idx,i in enumerate(prm_obj.atoms) if idx in solute_atoms ]

# use the dipole_moment function of mdtraj to compute the dipole moment components 
dip_moments = md.dipole_moments(solute_traj, solute_charges)
# obtain the dipole magnitude
dipole_magnitude = np.sqrt(np.square(dip_moments).sum(axis=1))

# calculate mean, standard deviation, and median of the dipole moment x-, y-, z- components
dip_components_av = list(dip_moments.mean(axis=0))  
dip_components_std = list(dip_moments.std(axis=0))  
dip_components_med = list(np.median(dip_moments, axis=0))  
# calculate mean, standard deviation, and median of the dipole magnitude
dip_mom_stats = ComposerGMX.get_stats(dipole_magnitude)  

# update the MDFP output dictionary and print out the dataframe
dict_dip_mom = {'av_mu_x': dip_components_av[0], 'av_mu_y': dip_components_av[1], 'av_mu_z': dip_components_av[2], 'std_mu_x': dip_components_std[0], 'std_mu_y': dip_components_std[1], 'std_mu_z': dip_components_std[2], 'med_mu_x': dip_components_med[0], 'med_mu_y': dip_components_med[1], 'med_mu_z': dip_components_med[2], 'av_mu': dip_mom_stats[0], 'std_mu': dip_mom_stats[1], 'med_mu': dip_mom_stats[2]}
Dict_Fingerprint.update(dict_dip_mom)
print(pd.DataFrame([Dict_Fingerprint]))


       cmpd_name                                             smiles  \
0  CHEMBL1077779  CCCCC1N(CC2CCCCC2)C(=O)OC12CC[NH+](C1(C)CCN(C(...   

           MW  HA_count  RB_count  N_count  O_count  F_count  P_count  \
0  538.400319        39         8        3        3        0        0   

   S_count  ...   av_mu_z  std_mu_x  std_mu_y  std_mu_z  med_mu_x  med_mu_y  \
0        0  ... -0.134923  0.399214  0.348376  0.347874  0.127883 -0.097114   

   med_mu_z     av_mu    std_mu    med_mu  
0 -0.164791  0.647369  0.091712  0.644671  

[1 rows x 60 columns]


As previously mentioned, there are a number of python packages which could be useful for the analysis of MD trajectories. Below, there is an example of how to use functions of pytraj. 

In [15]:
# Load pytraj and load the trajectory using the function pt.iterload() of pytraj.

import pytraj as pt
pytraj_traj = pt.iterload(traj_file, top=pdb_file)

Calculate the center of geometry of the solute:

In [16]:
centroid_lig = pt.center_of_geometry(pytraj_traj, mask=':LIG & !(@H=)')
centroid_lig

array([[21.60871897, 20.94846249, 21.79718036],
       [20.8523087 , 20.55487275, 21.30846247],
       [20.85846253, 20.70820625, 20.92282152],
       ...,
       [21.36230881, 20.61359078, 22.02077022],
       [21.33589864, 20.43359078, 22.11025752],
       [21.31282183, 20.6951293 , 22.12359068]])